# Prediction Model Quality

## Task Reuirements

As the input we have some prediction model of new stores which are going to be launched to our internet-market. 

We understand that the main goal is related to a big overall number of such stores so we have to sort it somehow. The model was deployed as the result of linear regression which was named a potential. It means that each existing store has its own score (potential) and its value schould be related to store gross merchandise volume for some period. 

In this task we're going to estimate the quality of this model.

## Data Preparation

In [83]:
import pandas as pd
import plotly.express as px
import warnings
from scipy import stats 

In [ ]:
import mysql.connector  
import csv

db = mysql.connector.connect(host="******",    # your host, usually localhost
                     user="root",         # your username
                     passwd="**********",  # your password
                     db="***")        # name of the data base

cursor = db.cursor()
cursor.execute(

"""
with orders 
as 
(
    select 
    o.store_id as store_id 
    , o.order_id as order_id
    , (coalesce( (o.order_cost_lcy, 0) * o.currency_rate) as order_sum 
    , datediff ( 
        coalesce( o.utc_created_dttm, o.utc_cancelled_dttm, o.utc_confirmed_dttm) 
        , min (p.utc_created_dttm) over (partition by store_id) 
    ) as order_day_number    
    , min (p.utc_created_dttm) over (partition by store_id) as store_firstly_created
    from data.orders_data as o 
    inner join data.store_data as p 
    on o.store_id = p.store_id
    where coalesce( o.utc_created_dttm, o.utc_cancelled_dttm, o.utc_confirmed_dttm) between MakeDate(2021, 120) and MakeDate(2021, 250)
)

select 
p.store_id as store_id
, p.order_day_number as order_day_number
, p.max_order_day_number as max_order_day_number
, p.store_firstly_created as store_firstly_created
, p.order_count_per_store as order_count_per_store
, p.orders_sum_per_store as orders_sum_per_store
, p.days_worked as days_worked
, t.medal as medal
, t.potential as potential
from 
(
    select 
    store_id
    , order_day_number
    , max_order_day_number
    , store_firstly_created
    , count(order_id) as order_count_per_store
    , sum(order_sum) as orders_sum_per_store
    , count(distinct order_day_number) as days_worked
    from 
    (
        select 
        store_id
        , order_id
        , order_sum
        , order_day_number
        , store_firstly_created
        , max(order_day_number) over (partition by store_id) as max_order_day_number
        from orders 
    )
    group by 
    store_id
    , order_day_number
    , max_order_day_number
    , store_firstly_created
) as p

left join 
(
    select distinct 
    sd.store_id as store_id
    , pd.potential as potential
    , pd.medal as medal
    , pd.pot_launched_dttm as pot_launched_dttm
    from data.potential_data as pd 
    inner join data.store_data as sd 
    on pd.id = sd.pot_id  
) as t 
on p.store_id = t.store_id and p.store_firstly_created = t.pot_launched_dttm
"""
)

result = cursor.fetchall()

fp = open('stores_frame.csv', 'w')
myFile = csv.writer(fp)
myFile.writerows(result)
fp.close()
db.close()

The sql-script result is in csv below.

In [72]:
final_dataframe = pd.read_csv('stores_frame.csv', sep = ";")
final_dataframe[:5]

,days_worked,order_day_number,store_firstly_created,max_order_day_number,order_count_per_store,orders_sum_per_store,medal,store_id,potential
0,39,0,10.08.2021 11:11,47,10,6342,BRONSE,6150407691,5.416806
1,39,1,10.08.2021 11:11,47,3,4145,BRONSE,6150407691,5.416806
2,39,6,10.08.2021 11:11,47,1,1299,BRONSE,6150407691,5.416806
3,39,7,10.08.2021 11:11,47,3,1978,BRONSE,6150407691,5.416806
4,39,8,10.08.2021 11:11,47,1,1189,BRONSE,6150407691,5.416806


## Some overall statistics of stores results for first 90 days of work by medal groups

In [73]:
def groups_and_stats(df_input, n):
    df_further = df_input.copy()
    df_further['day'] = n
    df = df_further.query('max_order_day_number >= day and order_day_number <= day')

    k = df.groupby('store_id').count()[['medal']].reset_index()
    k = k.rename(columns = {'medal': 'workdays'})   
    
    df_fin = pd.merge(df, k, how = 'left', left_on = ['store_id'], right_on = ['store_id'])    

    df_result = df_fin.groupby(['store_id', 'max_order_day_number', 'workdays', 'medal']).sum()[['order_count_per_store', 'orders_sum_per_store']]
    df_result = df_result.reset_index()

    
    df_stat = df_result.copy()
    df_stat['orders_per_day'] = df_stat['order_count_per_store'] / n
    df_stat['orders_per_int'] = df_stat['order_count_per_store'] / df_stat['workdays']
    df_stat['gmv_per_day'] = df_stat['orders_sum_per_store'] / n
    df_stat['gmv_per_int'] = df_stat['orders_sum_per_store'] / df_stat['workdays']
    
    df_stat_cnt = df_stat.groupby("medal").count().store_id.reset_index()

    df_stat_ord_mean = df_stat.groupby("medal").mean().order_count_per_store
    df_stat_ord_med = df_stat.groupby("medal").median().order_count_per_store

    df_stat_sum_mean = df_stat.groupby("medal").mean().orders_sum_per_store
    df_stat_sum_med = df_stat.groupby("medal").median().orders_sum_per_store

    df_stat_ord_day = df_stat.groupby("medal").mean().orders_per_day
    df_stat_ord_int = df_stat.groupby("medal").mean().orders_per_int

    df_stat_gmv_day = df_stat.groupby("medal").mean().gmv_per_day
    df_stat_gmv_int = df_stat.groupby("medal").mean().gmv_per_int

    df_stat_fin = pd.merge(df_stat_cnt, df_stat_ord_mean, how = 'left', left_on = ['medal'], right_on = ['medal'])
    df_stat_fin = pd.merge(df_stat_fin, df_stat_ord_med, how = 'left', left_on = ['medal'], right_on = ['medal'])
    df_stat_fin = pd.merge(df_stat_fin, df_stat_sum_mean, how = 'left', left_on = ['medal'], right_on = ['medal'])
    df_stat_fin = pd.merge(df_stat_fin, df_stat_sum_med, how = 'left', left_on = ['medal'], right_on = ['medal'])
    df_stat_fin = pd.merge(df_stat_fin, df_stat_ord_day, how = 'left', left_on = ['medal'], right_on = ['medal'])
    df_stat_fin = pd.merge(df_stat_fin, df_stat_ord_int, how = 'left', left_on = ['medal'], right_on = ['medal'])
    df_stat_fin = pd.merge(df_stat_fin, df_stat_gmv_day, how = 'left', left_on = ['medal'], right_on = ['medal'])
    df_stat_fin = pd.merge(df_stat_fin, df_stat_gmv_int, how = 'left', left_on = ['medal'], right_on = ['medal'])

    df_stat_fin = df_stat_fin.rename(columns = {'order_count_per_store_x': 'avg_orders_count', 'order_count_per_store_y': 'median_orders_count', 'orders_sum_per_store_x': 'avg_gmv', 'orders_sum_per_store_y': 'median_gmv'})    
    df_stat_fin["avg_orders_count"] = round(df_stat_fin["avg_orders_count"], 1)
    df_stat_fin["avg_gmv"] = round(df_stat_fin["avg_gmv"], 0)
    df_stat_fin["median_gmv"] = round(df_stat_fin["median_gmv"], 0)
    df_stat_fin["orders_per_day"] = round(df_stat_fin["orders_per_day"], 1)
    df_stat_fin["orders_per_int"] = round(df_stat_fin["orders_per_int"], 1)
    df_stat_fin["gmv_per_day"] = round(df_stat_fin["gmv_per_day"], 0)
    df_stat_fin["gmv_per_int"] = round(df_stat_fin["gmv_per_int"], 0)
    
    return df_result, df_stat_fin

In [74]:
warnings.filterwarnings("ignore")
df_sales = final_dataframe.copy()

#the number of unique stores
print(str(len(pd.unique(df_sales["store_id"]))))

1029


In [75]:
df_sales_90 = groups_and_stats(df_sales, 90)[0]
df_stats_90 = groups_and_stats(df_sales, 90)[1]
df_stats_90

,medal,store_id,avg_orders_count,median_orders_count,avg_gmv,median_gmv,orders_per_day,orders_per_int,gmv_per_day,gmv_per_int
0,BRONSE,181,108.5,58,110997.0,55775,1.2,1.5,1233.0,1555.0
1,GOLD,207,272.6,137,235604.0,117974,3.0,2.9,2618.0,2539.0
2,NO MEDAL,65,75.6,33,65560.0,20990,0.8,1.2,728.0,1022.0
3,SILVER,174,130.0,69,113981.0,63867,1.4,1.5,1266.0,1351.0


## Let's have a look 

In [58]:
def agg_with_pot(df_input, n):
    df_further = df_input.copy()
    df_further['day'] = n
    df = df_further.query('max_order_day_number >= day and order_day_number <= day')
    
    #строк на ресторан, т.е. сколько дней ресторан работал в этот интервал времени

    k = df.groupby('store_id').count()[['medal']].reset_index()
    k = k.rename(columns = {'medal': 'workdays'})   
    
    df_fin = pd.merge(df, k, how = 'left', left_on = ['store_id'], right_on = ['store_id'])    

    df_result = df_fin.groupby(['store_id', 'max_order_day_number', 'workdays', 'medal', 'potential']).sum()[['order_count_per_store', 'orders_sum_per_store']]
    df_result = df_result.reset_index()
    
    df_stat = df_result.copy()
    df_stat['orders_per_day'] = df_stat['order_count_per_store'] / n
    df_stat['orders_per_int'] = df_stat['order_count_per_store'] / df_stat['workdays']
    df_stat['gmv_per_day'] = df_stat['orders_sum_per_store'] / n
    df_stat['gmv_per_int'] = df_stat['orders_sum_per_store'] / df_stat['workdays']

    df_res = df_stat.copy()
    
    return df_res


In [81]:
def visual(df_input):

    x_pot = np.array(df_input["order_count_per_store"])
    y_pot = np.array(df_input["orders_sum_per_store"])
    fig_pot = px.scatter(x = x_pot, y = y_pot, labels = {'x': 'Stores orders', 'y': 'Stores GMV'} , color = df_input.medal, color_discrete_map = {'GOLD':'#FFD700', 'BRONSE': '#CD853F', 'SILVER': '#C0C0C0', 'NO MEDAL': '#FF4500', 'unknown': '#000000'},  title = "GMV-orders relation by medals")

    x_ord = np.array(df_input["potential"])
    y_ord = np.array(df_input["orders_sum_per_store"])
    fig_ord = px.scatter(x = x_ord, y = y_ord, labels = {'x': 'Potential', 'y': 'Stores GMV'} , color = df_input.medal, color_discrete_map = {'GOLD':'#FFD700', 'BRONSE': '#CD853F', 'SILVER': '#C0C0C0', 'NO MEDAL': '#FF4500', 'unknown': '#000000'},  title = "GMV-potential relation by medals")

    df_not_zero = df_input.query('potential > 0')
    x_log = np.array(df_not_zero["potential"].apply(np.log))
    y_log = np.array(df_not_zero["orders_sum_per_store"].apply(np.log))
    fig_log = px.scatter(x = x_log, y = y_log, labels = {'x': 'Potential', 'y': 'Stores GMV'} , color = df_not_zero.medal, color_discrete_map = {'GOLD':'#FFD700', 'BRONSE': '#CD853F', 'SILVER': '#C0C0C0', 'NO MEDAL': '#FF4500', 'unknown': '#000000'},  title = "GMV-potential logarithms relation by medals")

    x_int = np.array(df_input["potential"])
    y_int = np.array(df_input["gmv_per_int"])
    fig_int = px.scatter(x = x_int, y = y_int, labels = {'x': 'Potential', 'y': 'Stores GMV per workday'} , color = df_input.medal, color_discrete_map = {'GOLD':'#FFD700', 'BRONSE': '#CD853F', 'SILVER': '#C0C0C0', 'NO MEDAL': '#FF4500', 'unknown': '#000000'},  title = "Daily GMV-potential relation by medals")

    return fig_pot.show(), fig_ord.show(), fig_log.show(), fig_int.show()

In [82]:
n = 90
df_sales_90 = agg_with_pot(df_sales, n)
visual(df_sales_90)[0]

## Linear correlation 

As we know such attribute as potential is a result of some linear regression related to GMV values. We're about to find some correlation between those attributes using Pearson correlation coefficient for daily GMV and GMV by 90 days in metric and logarithmic scale both.

In [84]:
def pierson_corr(df):
    a = np.array(df['potential'])
    b = np.array(df['orders_sum_per_store'])
    
    if (stats.pearsonr(a, b)[0] > 0.7 or stats.pearsonr(a, b)[0] < -0.7) :
        st = "Potetnial-GMV correlation. The result is strong relationship: "
    elif ((stats.pearsonr(a, b)[0] > 0.3 and stats.pearsonr(a, b)[0] < 0.7) or (stats.pearsonr(a, b)[0] > -0.7 and stats.pearsonr(a, b)[0] < -0.3)):
        st = "Potetnial-GMV correlation. The result is moderate relationship: "
    elif ((stats.pearsonr(a, b)[0] > 0.1 and stats.pearsonr(a, b)[0] < 0.3) or (stats.pearsonr(a, b)[0] > -0.3 and stats.pearsonr(a, b)[0] < -0.1)):
        st = "Potetnial-GMV correlation. The result is weak relationship: "
    else:
        st = "Potetnial-GMV correlation. The result is no relationship: "
    
    df_not_zero = df.query('potential > 0')
    
    a_log = np.array(df_not_zero['potential'].apply(np.log))
    b_log = np.array(df_not_zero['orders_sum_per_store'].apply(np.log))

    if (stats.pearsonr(a_log, b_log)[0] > 0.7 or stats.pearsonr(a_log, b_log)[0] < -0.7) :
        st_log = "Potetnial-GMV log correlation. The result is strong relationship: "
    elif ((stats.pearsonr(a_log, b_log)[0] > 0.3 and stats.pearsonr(a_log, b_log)[0] < 0.7) or (stats.pearsonr(a_log, b_log)[0] > -0.7 and stats.pearsonr(a_log, b_log)[0] < -0.3)):
        st_log = "Potetnial-GMV log correlation. The result is moderate relationship: "
    elif ((stats.pearsonr(a_log, b_log)[0] > 0.1 and stats.pearsonr(a_log, b_log)[0] < 0.3) or (stats.pearsonr(a_log, b_log)[0] > -0.3 and stats.pearsonr(a_log, b_log)[0] < -0.1)):
        st_log = "Potetnial-GMV log correlation. The result is weak relationship: "
    else:
        st_log = "Potetnial-GMV log correlation. The result is no relationship: "
        
    return print( str(st) + str(stats.pearsonr(a, b)[0])), print( str(st_log) + str(stats.pearsonr(a_log, b_log)[0]))



#коэффициент корреляции пирсона - то же самое, но для ежедневного дохода
def pierson_corr_int(df):
    a = np.array(df['potential'])
    b = np.array(df['gmv_per_int'])
    
    if (stats.pearsonr(a, b)[0] > 0.7 or stats.pearsonr(a, b)[0] < -0.7) :
        st = "Potetnial-Daily GMV correlation. The result is strong relationship: "
    elif ((stats.pearsonr(a, b)[0] > 0.3 and stats.pearsonr(a, b)[0] < 0.7) or (stats.pearsonr(a, b)[0] > -0.7 and stats.pearsonr(a, b)[0] < -0.3)):
        st = "Potetnial-Daily GMV correlation. The result is moderate relationship: "
    elif ((stats.pearsonr(a, b)[0] > 0.1 and stats.pearsonr(a, b)[0] < 0.3) or (stats.pearsonr(a, b)[0] > -0.3 and stats.pearsonr(a, b)[0] < -0.1)):
        st = "Potetnial-Daily GMV correlation. The result is weak relationship: "
    else:
        st = "Potetnial-Daily GMV correlation. The result is no relationship: "
    
    df_not_zero = df.query('potential > 0')
    
    a_log = np.array(df_not_zero['potential'].apply(np.log))
    b_log = np.array(df_not_zero['gmv_per_int'].apply(np.log))

    if (stats.pearsonr(a_log, b_log)[0] > 0.7 or stats.pearsonr(a_log, b_log)[0] < -0.7) :
        st_log = "Potetnial-Daily GMV correlation. The result is strong relationship: "
    elif ((stats.pearsonr(a_log, b_log)[0] > 0.3 and stats.pearsonr(a_log, b_log)[0] < 0.7) or (stats.pearsonr(a_log, b_log)[0] > -0.7 and stats.pearsonr(a_log, b_log)[0] < -0.3)):
        st_log = "Potetnial-Daily GMV correlation. The result is moderate relationship: "
    elif ((stats.pearsonr(a_log, b_log)[0] > 0.1 and stats.pearsonr(a_log, b_log)[0] < 0.3) or (stats.pearsonr(a_log, b_log)[0] > -0.3 and stats.pearsonr(a_log, b_log)[0] < -0.1)):
        st_log = "Potetnial-Daily GMV correlation. The result is weak relationship: "
    else:
        st_log = "Potetnial-Daily GMV correlation. The result is no relationship: "
        
    return print( str(st) + str(stats.pearsonr(a, b)[0])), print( str(st_log) + str(stats.pearsonr(a_log, b_log)[0]))

In [85]:
pierson_corr(df_sales_90)[0]

Potetnial-GMV correlation. The result is weak relationship: 0.288425056955411
Potetnial-GMV log correlation. The result is weak relationship: 0.2070833720585255


In [86]:
pierson_corr_int(df_sales_90)[0]

Potetnial-Daily GMV correlation. The result is weak relationship: 0.29556654330942544
Potetnial-Daily GMV correlation. The result is weak relationship: 0.19954931410015636


## Classification model 

As we could see the potential score doesn't work as it was planned. Probably we can consider it like a binary classification model: in case when linear regression result about zero it should be really bad fact result in terms of GMV. Else we can't predict.

In this case we consider predictive 1 and 0 according to store's medals and fact 1 and 0 should be related to some results of stores which were launched previously. We will get median, average and some percentiles of GMV to compare old and new stores.

In [109]:
def limit_gmv_gold_silv(df_input, limit):

    df_input["medal"] = df_input["medal"].fillna('unknown')
    df_input = df_input.query('medal != "unknown"')
    df_input = df_input.sort_values(by = "orders_sum_per_store", ascending = False).reset_index()

    df_input["predict"] = 0
    df_input["fact"] = 0
    
    for i in range(df_input.shape[0]):

        if (df_input.at[i, "medal"] == "GOLD" or df_input.at[i, "medal"] == "SILVER"):
            df_input.at[i, "predict"] = 1
        else:
            df_input.at[i, "predict"] = 0

        if df_input.at[i, "orders_sum_per_store"] > limit:
            df_input.at[i, "fact"] = 1
        else:
            df_input.at[i, "fact"] = 0

    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(df_input.shape[0]):
        if df_input.at[i, "predict"] == 1 and df_input.at[i, "fact"] == 1:
            TP = TP + 1
        if df_input.at[i, "predict"] == 1 and df_input.at[i, "fact"] == 0:
            FP = FP + 1
        if df_input.at[i, "predict"] == 0 and df_input.at[i, "fact"] == 0:
            TN = TN + 1
        if df_input.at[i, "predict"] == 0 and df_input.at[i, "fact"] == 1:
            FN = FN + 1

    precision = round((TP / (TP + FP)), 2)
    recall = round((TP / (TP + FN)),2)
    b = 1
    F = round(((1 + b * b) * ((b * b) * precision * recall) / (precision + recall)),2)
    FPR = round(((FP / (FP + TN))),2)
    
    return print ( ("True Positive: " + str(TP) + " False Positive: " + str(FP) + " True Negative: " + str(TN) + " False Negative: " + str(FN)) ), print( ("precision: " + str(precision)) ), print("recall: " + str(recall)), print( ("F-score: ") + (str(F)), print( ("FPR: ") + str(FPR) )  )

In [110]:
keys = ['average', 'median', 'top10percent', 'top15percent', 'top20percent', 'top25percent']
limits_90 = {'average': 102704, 'median': 39680, 'top10percent': 228787, 'top15percent': 165192, 'top20percent': 128303, 'top25percent': 101508}

n = 90
limits = limits_90
df_pre_rec = groups_and_stats(df_sales, n)[0]

for i in keys: 
    print (str(i) + " value " + str(limits[i]))
    print(limit_gmv_gold_silv(df_pre_rec, limits[i])[0])
    print("")


average value 102704
True Positive: 172 False Positive: 209 True Negative: 178 False Negative: 68
precision: 0.45
recall: 0.72
FPR: 0.54
F-score: 0.55 None
None

median value 39680
True Positive: 260 False Positive: 121 True Negative: 114 False Negative: 132
precision: 0.68
recall: 0.66
FPR: 0.51
F-score: 0.67 None
None

top10percent value 228787
True Positive: 95 False Positive: 286 True Negative: 217 False Negative: 29
precision: 0.25
recall: 0.77
FPR: 0.57
F-score: 0.38 None
None

top15percent value 165192
True Positive: 121 False Positive: 260 True Negative: 204 False Negative: 42
precision: 0.32
recall: 0.74
FPR: 0.56
F-score: 0.45 None
None

top20percent value 128303
True Positive: 151 False Positive: 230 True Negative: 193 False Negative: 53
precision: 0.4
recall: 0.74
FPR: 0.54
F-score: 0.52 None
None

top25percent value 101508
True Positive: 172 False Positive: 209 True Negative: 176 False Negative: 70
precision: 0.45
recall: 0.71
FPR: 0.54
F-score: 0.55 None
None



## Short conclusions

- There is weak correlation between potential and GMV (or orders) results of stores. Primary predictive logic doesn't work.
- Nevertheless we can notice quite high rates of recall (> 0.7) it terms of using such model as classifications. Basically it means that attributes used in primary linear regression are not enough for store result defining. But when such attributes are about zero the probability of "bad" results (in terms of average / median results of existing stores) is high.
- Due to large number of stores in area we should concentrate on recall growth.